# Global Centroid Moment Tensor 3D (GCMT3D)

## The workflow

Make sure you are in the right virtual environment.

### Preparing Specfem3d_globe

Before we create or add an earthquake to the database, the `Par_file` 
that sets all parameters for specfem needs to be set. Meaning that the 
All parameters, such as resolution, length of simulation, computational 
setup, etc., need to configured within the parameter file so that we can 
request the right number resources ono the fly. This is done using the `DATA
fixer` class. After the `DATAFixer` class has modified the Par_file. It can be
called to run the mesher. Note that for the following steps `specfem3d_globe`
already has to be compiled.

In [1]:
from gcmt3d.runSF3D.runSF3D import DATAFixer
import os

SPECFEM_DIR = "/scratch/gpfs/lsawade/specfem_tutorial/specfem3d_globe"

# Set MPI Parameters
NEX_XI = 128
NEX_ETA = 128
NPROC_XI = 2
NPROC_ETA = 2
GPU_MODE = False
ADIOS_ENABLED = False

# Minimal Processing
ROTATE_SEISMOGRAMS_RT = True
RECORD_LENGTH = 60
MODEL = "s362ani"

# Output style
WRITE_SEISMOGRAMS_BY_MASTER = True
OUTPUT_SEISMOS_ASCII_TEXT = False
OUTPUT_SEISMOS_SAC_ALPHANUM = False
OUTPUT_SEISMOS_SAC_BINARY = True
OUTPUT_SEISMOS_ASDF = False

# Visualization
MOVIE_SURFACE = False
MOVIE_VOLUME = False
MOVIE_COARSE = False

# Computational Resources
nodes = 1
tasks = 24
walltime = "00:30:00"
# If solver is supposed to be used set these values. NOT the correct
# solver wrapper for the CMT inversion.
walltime_solver = None,
nodes_solver = None,
tasks_solver = None,

# Verbose output
verbose = True



DF = DATAFixer(SPECFEM_DIR,
               NEX_XI=NEX_XI, NEX_ETA=NEX_ETA, 
               NPROC_XI=NPROC_XI, NPROC_ETA=NPROC_ETA,
               ROTATE_SEISMOGRAMS_RT=ROTATE_SEISMOGRAMS_RT,
               RECORD_LENGTH=RECORD_LENGTH, MODEL=MODEL,
               WRITE_SEISMOGRAMS_BY_MASTER=WRITE_SEISMOGRAMS_BY_MASTER,
               OUTPUT_SEISMOS_ASCII_TEXT=OUTPUT_SEISMOS_ASCII_TEXT,
               OUTPUT_SEISMOS_SAC_ALPHANUM=OUTPUT_SEISMOS_SAC_ALPHANUM,
               OUTPUT_SEISMOS_SAC_BINARY=OUTPUT_SEISMOS_SAC_BINARY,
               OUTPUT_SEISMOS_ASDF=OUTPUT_SEISMOS_ASDF,
               MOVIE_SURFACE=MOVIE_SURFACE,
               MOVIE_VOLUME=MOVIE_VOLUME,
               MOVIE_COARSE=MOVIE_COARSE,
               nodes=nodes, tasks=tasks, walltime=walltime,
               nodes_solver=nodes_solver, tasks_solver=tasks_solver, walltime_solver=walltime_solver,
               verbose=verbose)

# Run `Par_file` fixer.
DF.fix_parfiles()

After the `Par_file` is fixed, we can run the mesher. The mesher 
only has to be run once if the only thing that changes are the 
sources and the receivers and the CMT Solution. In this case, 
it's only the CMT solution that changes. 

In [2]:
# Run the mesher to a slurm scheduler.
DF.run_mesher()

sbatch -N 1 -n 24 -D /scratch/gpfs/lsawade/specfem_tutorial/specfem3d_globe -t 00:30:00 /scratch/gpfs/lsawade/GCMT3D/gcmt3d/runSF3D/batch/mesh.sbatch
Command has been sent.
Output:
 Submitted batch job 3093815

Errors:
 


### Creating the Database Skeleton

The `DatabaseSkeleton` `class` handles the setup of the database 
and creates new directories for new CMTs.

```bash
database
├── eq_<eq_ID_1>
│   ├── CMT_SIMs/
squeue
│   │   ├── CMT_depth/
│   │   │    ├── DATA/
│   │   │    ├── DATABASES_MPI/
│   │   │    ├── OUTPUFILES/
│   │   │    └── bin/        # symbolic link --> SPECFEM_DIR/bin/
│   │   ├── CMT_lat/
│   │   │    ├── DATA/
│   │   │    :
│   │   ├── CMT_lon/
│   │   ├── CMT_pp/
│   │   ├── CMT_rp/
│   │   ├── CMT_rr/
│   │   ├── CMT_rt/
│   │   ├── CMT_tp/
│   │   └── CMT_tt/
│   ├── eq_9703873.cmt
│   ├── station_data/
│   ├── window_data/
│   └── seismograms/
│
├── eq_<eq_ID_2>/
│   ├── CMT_SIMs/
│   :
│
├── eq_<eq_ID_3>/
:

```

In [1]:
from gcmt3d.data.management.skeleton import DataBaseSkeleton


# Set database directory
databasedir = "../../test_sample_database"
cmt_filename = "../../cmtgermany.cmt"

# Create Database skeleton class. Note that the specfem directory is not set
# because it is not available on every system.
DB = DataBaseSkeleton(basedir=databasedir, cmt_fn=cmt_filename, specfem_dir=SPECFEM_DIR, verbose=True, overwrite=False)

NameError: name 'SPECFEM_DIR' is not defined

In [5]:
# Create database structure
DB.create_all()

../../test_sample_database/eq_120504B exists already. Not overwritten.
Earthquake file ../../test_sample_database/eq_120504B/eq_120504B.cmt exists already. It will NOT be overwritten.
../../test_sample_database/eq_120504B/station_data exists already. Not overwritten.
../../test_sample_database/eq_120504B/window_data exists already. Not overwritten.
../../test_sample_database/eq_120504B/seismograms exists already. Not overwritten.
../../test_sample_database/eq_120504B/CMT_SIMs exists already. Not overwritten.
../../test_sample_database/eq_120504B/CMT_SIMs/CMT_rr exists already. Not overwritten.
Directory ../../test_sample_database/eq_120504B/CMT_SIMs/CMT_rr/DATA exists already. It will NOT be overwritten.
Directory ../../test_sample_database/eq_120504B/CMT_SIMs/CMT_rr/DATABASES_MPI exists already. It will NOT be overwritten.
Directory ../../test_sample_database/eq_120504B/CMT_SIMs/CMT_rr/OUTPUT_FILES exists already. It will NOT be overwritten.
../../test_sample_database/eq_120504B/CMT_S

### Downloading the Data

The first step in the inversion process is downloading data that corresponds to the original solution of Global CMT Catalogue, which is maintained by the Lamont-Doherty Earth Observatory.

To do this we use the Specifically design DataRequest class. We have to define certain parameters before we download the data:

In [2]:
from gcmt3d.data import DataRequest
from gcmt3d.source import CMTSource
import os

# Parameters
duration = 3600      # seconds, 1h = 3600s
channels = ['BHN', 'BHE', 'BHZ']   # needs to be a list
locations=['00']     # location 00 is the primary location for most seismometers
starttime_offset=-60 # -60 seconds is 1 min before the earthquakes inverted cmt solution 
resp_format="resp"   # response format


# Earthquake ID
eqID = CMTSource.from_CMTSOLUTION_file(cmt_filename).eventname
outputdir = os.path.join(databasedir, "eq_" + eqID)  # specific output directory here left open 
                                                    # if left open files will be saved in the 
                                                    # folder wher the `.cmt` file is located

# Earthquake and Station parameters
cmt_filename = os.path.join(outputdir, os.path.basename(outputdir)+".cmt")
stationlist_filename = None

# Create Request Object   
Request = DataRequest.from_file(cmt_filename,
                                stationlistfname=stationlist_filename,
                                duration=duration,
                                channels=channels,
                                locations=locations,
                                starttime_offset=starttime_offset,
                                outputdir=outputdir)

# Print Earthquake Download Info
print(Request)


Earthquake Parameters
--------------------------------------------------
Earthquake ID: 120504B
Origin Time: 2004-12-05T01:52:41.100000Z
Origin Latitude: 48.03
Origin Longitude: 8.04

Download Parameters:
--------------------------------------------------
Starttime: 2004-12-05T01:51:41.100000Z
Endtime: 2004-12-05T02:51:41.100000Z
Duration [s]: 3600
Channels: ['BHN', 'BHE', 'BHZ']
Locations: ['00']

Saving Parameters:
--------------------------------------------------
Output Directory: ../../test_sample_database/eq_120504B



After setting up the Request and inspecting the parameters we can download the data:

In [3]:
# Download the data using the download method
Request.download()

Reading data selection from list file '../../test_sample_database/eq_120504B/request.txt'
Received 7.7 KB of metadata in 0.5 seconds (16.6 KB/s)
Processed metadata for 317 channel epochs in 0.5 seconds
Received 24.5 MB of time series data in 18.6 seconds (1.3 MB/s) - written to ../../test_sample_database/eq_120504B/seismograms/obs/120504B.mseed
Received 271.0 KB of StationXML in 5.5 seconds (49.7 KB/s)
DONE at 2019-06-03 16:48:33


The next step is to create a STATIONS file, which tells specfem3d_globe where to "record" the simulated traces:

In [4]:
# Creating STATIONS file
Request.specfem_list()

# Move the STATIONS file to the subdirectory and overwrite the original ones..




['IU', 'AAE', '9.0292', '38.7656', '2442.0', '0.0']
['II', 'AAK', '42.6390', '74.4940', '1645.0', '30.0']
['II', 'ABKT', '37.9304', '58.1189', '678.0', '7.0']
['II', 'ABPO', '-19.0180', '47.2290', '1528.0', '5.3']
['IU', 'ADK', '51.8823', '-176.6842', '130.0', '0.0']
['IU', 'AFI', '-13.9093', '-171.7773', '705.0', '1.0']
['II', 'ALE', '82.5033', '-62.3500', '60.0', '0.0']
['IU', 'ANMO', '34.9459', '-106.4572', '1750.0', '100.0']
['IU', 'ANTO', '39.8680', '32.7934', '1090.0', '195.0']
['CU', 'ANWB', '17.6685', '-61.7856', '39.0', '0.0']
['II', 'ARU', '56.4302', '58.5625', '250.0', '0.0']
['II', 'ASCN', '-7.9327', '-14.3601', '173.0', '100.0']
['CU', 'BBGH', '13.1434', '-59.5588', '180.0', '0.0']
['IU', 'BBSR', '32.3713', '-64.6963', '-1.3', '31.4']
['CU', 'BCIP', '9.1665', '-79.8373', '61.0', '0.0']
['GT', 'BDFB', '-15.6418', '-48.0148', '1095.0', '100.0']
['II', 'BFO', '48.3319', '8.3311', '589.0', '0.0']
['GT', 'BGCA', '5.1764', '18.4242', '576.0', '100.0']
['IU', 'BILL', '68.0653', '

## Setting up specfem3d_globe

To run specfem in an automated fashion, we need the derivative information of the traces with respect to the moment tensor attributes as well as the centroid location.
I import the values that Wenjie set in his inversion.


In [8]:
# Import pycmt3D values!
from pycmt3d.constant import SCALE_MOMENT, SCALE_DEPTH, SCALE_LONGITUDE, SCALE_LATITUDE
from gcmt3d.source import CMTSource
from gcmt3d.data import SpecfemSources
import os


# Basic parameters
cmt = CMTSource.from_CMTSOLUTION_file(cmt_filename)
npar = 9
dm = SCALE_MOMENT       # 10**22 dyne*cm
dx = SCALE_DEPTH        # 1000 m
ddeg = SCALE_LATITUDE   # 0.001 deg
outdir = os.path.join(outputdir, "CMT_SIMs")

sfsource = SpecfemSources(cmt, npar=npar, dm=dm, dx=dx, ddeg=ddeg,
                          outdir=outdir)
sfsource.write_sources()



## Running SpecFEM3D Globe

Now that everything is in place. We can run specfem using the `runSF3D` `class`. Buuuut I need to fix the `STATIONS` file first. it needs to be replaced.

In [9]:
from gcmt3d.runSF3D.runSF3D import RunSimulation


# These are nodes per job for each simulation, this a different run
nodes = 1
tasks = 24
walltime = "04:00:00"
RD = RunSimulation(os.path.abspath(outputdir), N=nodes, n=tasks, walltime=walltime, verbose=True)

# Replace the original STATIONSFILE with a new one
# RD.replace_STATIONS(os.path.join(os.path.abspath(outputdir), "station_data", "STATIONS"))



### Start the simulations

Now, this will take a minimum amount of time to run, but notice that the simulation is only submitted to the scheduler. You will have to wait until it's done to continue with processing.

In [10]:
RD()

CompletedProcess(args=['/scratch/gpfs/lsawade/GCMT3D/gcmt3d/runSF3D/batch/drive.sh', '1', '24', '9', '/scratch/gpfs/lsawade/sample_database/eq_120504B/CMT_SIMs', '04:00:00', '1', '/scratch/gpfs/lsawade/GCMT3D/gcmt3d/runSF3D/batch/drive.sbatch'], returncode=0, stdout='1\n24\n0-8\n/scratch/gpfs/lsawade/sample_database/eq_120504B/CMT_SIMs\n04:00:00\n/scratch/gpfs/lsawade/GCMT3D/gcmt3d/runSF3D/batch/drive.sbatch\nSUBMITTING ...\nSubmitted batch job 3093828\nSUBMITTED.\n', stderr='')
/scratch/gpfs/lsawade/GCMT3D/gcmt3d/runSF3D/batch/drive.sh 1 24 9 /scratch/gpfs/lsawade/sample_database/eq_120504B/CMT_SIMs 04:00:00 1 /scratch/gpfs/lsawade/GCMT3D/gcmt3d/runSF3D/batch/drive.sbatch
Command has been sent.
Output:
 1
24
0-8
/scratch/gpfs/lsawade/sample_database/eq_120504B/CMT_SIMs
04:00:00
/scratch/gpfs/lsawade/GCMT3D/gcmt3d/runSF3D/batch/drive.sbatch
SUBMITTING ...
Submitted batch job 3093828
SUBMITTED.

Errors:
 


'/scratch/gpfs/lsawade/GCMT3D/notebooks'